In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob
import sys
import numpy
import tsfel
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tcn import TCN, tcn_full_summary
from tcn import compiled_tcn
from mango.tuner import Tuner
from keras_flops import get_flops
import pickle
import csv
import time
import matplotlib.pyplot as plt
import itertools
import random
from hardware_utils import *

In [ ]:
NAS_EPOCHS = 100 #NAS epochs
EPOCHS = 150 #model epochs
device = "NUCLEO_L4R5ZI_P" #hardware name
dirpath='FeatNN_Mbed_Prog/' #mbed program directory
model_name = "featNN"+device+"_"+".h5"
os.system("mkdir -p trained_models/")
platform_connected = False #HIL or proxy

log_file_name = 'log_NAS_featNN'+"_"+device+'.csv' #log file for NAS
if os.path.exists(log_file_name):
    os.remove(log_file_name)
row_write = ['score', 'accuracy','SRAM','Flash','Latency',
             'nb_filters','kernel_size','dilations','nb_stacks','use_skip_connections',
            'use_iqr','use_max','use_Median','use_variance','use_MAD','use_Abs_energy',
            'use_entropy','use_ppd','use_fft','use_ff','use_mps']
with open(log_file_name, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_write)
if os.path.exists(log_file_name[0:-4]+'.p'):
    os.remove(log_file_name[0:-4]+'.p')

## Import Dataset

In [ ]:
x_train_sig = np.loadtxt('UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt', dtype='float32')
X_train_sig = pd.DataFrame(np.hstack(x_train_sig), columns=["total_acc_x"])
Y_train = np.loadtxt('UCI HAR Dataset/train/y_train.txt')
Y_train = Y_train-1
Y_train = to_categorical(Y_train.reshape((len(Y_train),1)),num_classes=len(set(Y_train)))

## Training and NAS

In [ ]:
def objective_NN(nb_filters=6,kernel_size=32,dilations=[1,2,4,8,16,32,64,128],
                 nb_stacks=1, use_skip_connections = True,
                use_iqr = True, use_max = True, use_Median= True,
                use_variance = True, use_MAD = True,
                use_Abs_energy = True, use_entropy = True,
                use_ppd = True, use_fft = True, use_ff = True, use_mps = True):
    
    feat_mask = [0,0,0,0,0,0,0,0,0,0,0,0]
    stat_feat_list = []
    temporal_feat_list = []
    spectral_feat_list = []
    
    if(use_fft==True):
        spectral_feat_list.append('FFT mean coefficient')
        feat_mask[0] = 1
    if(use_ff==True):
        spectral_feat_list.append('Fundamental frequency') 
        feat_mask[1] = 1
    if(use_mps==True):
        spectral_feat_list.append('Max power spectrum')  
        feat_mask[2] = 1
    if(use_iqr==True):
        stat_feat_list.append('Interquartile range')
        feat_mask[3] = 1
    stat_feat_list.append('Mean')
    feat_mask[4] = 1
    if(use_max==True):
        stat_feat_list.append('Max')
        feat_mask[5] = 1
    if(use_Median==True):
        stat_feat_list.append('Median')
        feat_mask[6] = 1
    if(use_variance==True):
        stat_feat_list.append('Variance')  
        feat_mask[7] = 1
    if(use_MAD==True):
        stat_feat_list.append('Mean absolute deviation')
        feat_mask[8] = 1
    if(use_Abs_energy==True):
        temporal_feat_list.append('Absolute energy')
        feat_mask[9] = 1
    if(use_entropy==True):
        temporal_feat_list.append('Entropy') 
        feat_mask[10] = 1
    if(use_ppd==True):
        temporal_feat_list.append('Peak to peak distance')  
        feat_mask[11] = 1
    
    feat_mask = '{'+str(feat_mask)[1:-1]+'}'
 
    cgf_file = tsfel.get_features_by_domain()
    for item in cgf_file['statistical'].keys():
        if(item not in stat_feat_list):
            cgf_file['statistical'][item]['use'] = 'no'
    for item in cgf_file['spectral'].keys():
        if(item=='FFT mean coefficient'):
            cgf_file['spectral'][item]['parameters']['nfreq'] = 64
        if(item not in spectral_feat_list):
            cgf_file['spectral'][item]['use'] = 'no'
    for item in cgf_file['temporal'].keys():
        if(item not in temporal_feat_list):
            cgf_file['temporal'][item]['use'] = 'no'

    X_train = tsfel.time_series_features_extractor(cgf_file, X_train_sig, fs=50, window_size=128)
    ################################################################################
    
    print(nb_filters,kernel_size,dilations,nb_stacks, use_skip_connections)
    training_flag = 0
    score = -5.0
    accuracy = -1.0
    inputs = layers.Input((X_train.shape[1]))
    x = inputs
    x = layers.Reshape((X_train.shape[1],1))(x)
    x = TCN(return_sequences=False,
                     nb_filters=nb_filters,
                     kernel_size=kernel_size,
                     dilations=dilations,
                     nb_stacks=nb_stacks,
                     use_skip_connections=use_skip_connections)(x)
    outputs = layers.Dense(Y_train.shape[1], activation="softmax", name="pred")(x)
    model = Model(inputs = inputs, outputs = outputs)
    opt = tf.keras.optimizers.Adam()
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=opt,metrics=['accuracy'])
    checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', verbose=1, save_best_only=True)
    ################################################################################
    if(platform_connected == False):
        Latency = get_flops(model, batch_size=1) #latency proxy
        RAM = get_model_memory_usage(batch_size=1, model=model) #SRAM proxy
        if(quantization==True):
            RAM = RAM/8.0
        Flash =  get_model_flash_usage(model,'g_featnn_model_data',quantization=quantization) #flash proxy
        if(RAM < maxRAM and Flash < maxFlash):
            training_flag = 1
        else:
            training_flag = 0
            score = -5.0
            accuracy = -1.0
    else:
        RAM, Flash, Latency, err_flag = platform_in_the_loop_controller(model,'g_featnn_model_data', 
                                            device,X_train.shape[1],128,50,feat_mask,
                                            dir_path=dirpath)
        if(RAM!=-1 and Flash!=-1 and Latency!=-1):
            training_flag = 1
        else:
            training_flag = 0
            score = -5.0
            accuracy = -1.0
    ################################################################################
    if(training_flag == 1):
        history = model.fit(x=X_train, y=Y_train,validation_split=0.1,
              epochs=EPOCHS,callbacks=[checkpoint],shuffle=True,verbose=1)
        accuracy = checkpoint.best
        score = accuracy + 0.01*((RAM/maxRAM) + (Flash/maxFlash)) +  0.01*(Latency/1e6)
    
    row_write = [score, accuracy,RAM,Flash,Latency,
             nb_filters,kernel_size,dilations,nb_stacks,use_skip_connections,
            use_iqr,use_max,use_Median,use_variance,use_MAD,use_Abs_energy,
            use_entropy,use_ppd,use_fft,use_ff,use_mps]    
    print('Design choice:',row_write)
    with open(log_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_write)     
    
    return score

In [ ]:
def save_res(data, file_name):
    pickle.dump( data, open( file_name, "wb" ) )
    
min_layer = 3
max_layer = 8
a_list = [1,2,4,8,16,32,64,128]
all_combinations = []
dil_list = []
for r in range(len(a_list) + 1):
    combinations_object = itertools.combinations(a_list, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list
all_combinations = all_combinations[1:]
for item in all_combinations:
    if(len(item) >= min_layer and len(item) <= max_layer):
        dil_list.append(list(item))

param_dict = {
    'nb_filters': range(3,64),
    'nb_stacks': [1,2,3],
    'kernel_size': range(3,16),
    'use_skip_connections': [True, False],
    'dilations': dil_list,
    'use_iqr' : [True, False],
    'use_max': [True, False],
    'use_Median': [True, False],
    'use_variance': [True, False],
    'use_MAD': [True, False],
    'use_Abs_energy': [True, False],
    'use_entropy': [True, False],
    'use_ppd': [True, False],
    'use_fft': [True, False],
    'use_ff': [True, False],
    'use_mps': [True, False]
}

def objfunc(args_list):

    objective_evaluated = []
    
    start_time = time.time()
    
    for hyper_par in args_list:
        nb_filters = hyper_par['nb_filters']
        nb_stacks = hyper_par['nb_stacks']
        kernel_size = hyper_par['kernel_size']
        use_skip_connections = hyper_par['use_skip_connections']
        dilations = hyper_par['dilations']
        
        use_iqr = hyper_par['use_iqr']
        use_max = hyper_par['use_max']
        use_Median = hyper_par['use_Median']
        use_variance = hyper_par['use_variance']        
        use_MAD = hyper_par['use_MAD']
        use_Abs_energy = hyper_par['use_Abs_energy']            
        use_entropy = hyper_par['use_entropy']
        use_ppd = hyper_par['use_ppd']    
        use_fft = hyper_par['use_fft']            
        use_ff = hyper_par['use_ff']
        use_mps = hyper_par['use_mps'] 
            
        objective = objective_NN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,
                 nb_stacks=nb_stacks, use_skip_connections = use_skip_connections, 
                use_iqr = use_iqr, use_max = use_max, use_Median= use_Median,
                use_variance = use_variance, use_MAD = use_MAD,
                use_Abs_energy = use_Abs_energy, use_entropy = use_entropy,
                use_ppd = use_ppd, use_fft = use_fft, use_ff = use_ff, use_mps = use_mps)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        
    return objective_evaluated

In [ ]:
conf_Dict = dict()
conf_Dict['batch_size'] = 1 
conf_Dict['num_iteration'] = NAS_EPOCHS
conf_Dict['initial_random']= 5
tuner = Tuner(param_dict, objfunc,conf_Dict)
all_runs = []
results = tuner.maximize()
all_runs.append(results)
save_res(all_runs,log_file_name[0:-4]+'.p')

## Train the best model and evaluate

In [ ]:
bool_ar = [results['best_params']['use_skip_connections'],
           results['best_params']['use_iqr'],
           results['best_params']['use_max'],
           results['best_params']['use_Median'],
           results['best_params']['use_variance'],
           results['best_params']['use_MAD'],
           results['best_params']['use_Abs_energy'],
           results['best_params']['use_entropy'],
           results['best_params']['use_ppd'],
           results['best_params']['use_fft'],
           results['best_params']['use_ff'],
           results['best_params']['use_mps']]

nb_filters=results['best_params']['nb_filters']
kernel_size=results['best_params']['kernel_size']
dilations=results['best_params']['dilations']
nb_stacks=results['best_params']['nb_stacks']
use_skip_connections = bool_ar[0]
use_iqr = bool_ar[1]
use_max = bool_ar[2]
use_Median= bool_ar[3]
use_variance = bool_ar[4]
use_MAD = bool_ar[5]
use_Abs_energy = bool_ar[6]
use_entropy = bool_ar[7]
use_ppd = bool_ar[8] 
use_fft = bool_ar[9]
use_ff = bool_ar[10] 
use_mps = bool_ar[11]
    
stat_feat_list = []
temporal_feat_list = []
spectral_feat_list = []
stat_feat_list.append('Mean')
if(use_iqr==True):
    stat_feat_list.append('Interquartile range')
if(use_max==True):
    stat_feat_list.append('Max')
if(use_Median==True):
    stat_feat_list.append('Median')
if(use_variance==True):
    stat_feat_list.append('Variance')    
if(use_MAD==True):
    stat_feat_list.append('Mean absolute deviation')
if(use_Abs_energy==True):
    temporal_feat_list.append('Absolute energy')
if(use_entropy==True):
    temporal_feat_list.append('Entropy') 
if(use_ppd==True):
    temporal_feat_list.append('Peak to peak distance')     
if(use_fft==True):
    spectral_feat_list.append('FFT mean coefficient')
if(use_ff==True):
    spectral_feat_list.append('Fundamental frequency') 
if(use_mps==True):
    spectral_feat_list.append('Max power spectrum')      

cgf_file = tsfel.get_features_by_domain()
for item in cgf_file['statistical'].keys():
    if(item not in stat_feat_list):
        cgf_file['statistical'][item]['use'] = 'no'
for item in cgf_file['spectral'].keys():
    if(item not in spectral_feat_list):
        cgf_file['spectral'][item]['use'] = 'no'
for item in cgf_file['temporal'].keys():
    if(item not in temporal_feat_list):
        cgf_file['temporal'][item]['use'] = 'no'

X_train = tsfel.time_series_features_extractor(cgf_file, X_train_sig, fs=50, window_size=128)
inputs = layers.Input((X_train.shape[1]))
x = inputs
x = layers.Reshape((X_train.shape[1],1))(x)
x = TCN(return_sequences=False,
                 nb_filters=nb_filters,
                 kernel_size=kernel_size,
                 dilations=dilations,
                 nb_stacks=nb_stacks,
                 use_skip_connections=use_skip_connections)(x)
outputs = layers.Dense(Y_train.shape[1], activation="softmax", name="pred")(x)
model = Model(inputs = inputs, outputs = outputs)
opt = tf.keras.optimizers.Adam()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
          optimizer=opt,metrics=['accuracy'])
checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', verbose=1, save_best_only=True)

history = model.fit(x=X_train, y=Y_train,validation_split=0.1,
      epochs=EPOCHS,callbacks=[checkpoint],shuffle=True,verbose=1)


In [ ]:
model = tf.keras.models.load_model(model_name,custom_objects={'TCN':TCN})
x_test_sig = np.loadtxt('UCI HAR Dataset/test/Inertial Signals/total_acc_x_test.txt', dtype='float32')
X_test_sig = pd.DataFrame(np.hstack(x_test_sig), columns=["total_acc_x"])
Y_test = np.loadtxt('UCI HAR Dataset/test/y_test.txt')
Y_test = Y_test-1
Y_test = to_categorical(Y_test.reshape((len(Y_test),1)),num_classes=len(set(Y_test)))
X_test = tsfel.time_series_features_extractor(cgf_file, X_test_sig, fs=50, window_size=128)
score = model.evaluate(X_test, Y_test, verbose=0)
score
